# FrankPEPstein: Interactive Peptide Fragment Design

This notebook implements the FrankPEPstein pipeline for designing peptide fragments binding to a specific protein pocket.

**Steps:**
1.  Setup & Dependencies
2.  Input Data (Receptor)
3.  Pocket Selection
4.  Fragment Generation & Ranking


## 0. Setup & Dependencies

In [ ]:
#@title Setup Pipeline (Run This First)
#@markdown This cell clones the repository, installs all dependencies (Mamba, CondaColab), and sets up the external tools.
#@markdown

import os
import sys
import subprocess

# --- 1. Clone Repository ---
if not os.path.exists("FrankPEPstein"):
    print("Cloning repository...")
    !git clone https://github.com/Joacaldog/FrankPEPstein.git
else:
    print("Repository already exists.")

# --- 2. Install CondaColab ---
try:
    import condacolab
except ImportError:
    print("Installing CondaColab...")
    !pip install -q condacolab
    import condacolab
condacolab.install()

# --- 3. Install Core Bio-Dependencies ---
print("Installing bio-dependencies (this may take a few minutes)...")
!mamba install -q -c conda-forge openbabel biopython fpocket joblib tqdm py3dmol salilab::modeller

# --- 4. Setup External Tools & Config ---
repo_path = os.path.abspath("FrankPEPstein")
if repo_path not in sys.path:
    sys.path.append(repo_path)
from scripts import notebook_utils

# DRIVE CONFIGURATION: Enter your File IDs here
drive_ids = {
    "adfr_id": "1gmRj8mva84-JB7UXUcQfB3Ziw_nwwdox",       # ADFRsuite_x86_64Linux_1.0.tar.gz
    "db_id": "1a4GoZ1ZT-DNYMyvVtKJukNdF6TAaLJU5",    # minipockets_..._curated-db.tar.gz
    "dict_id": "1nrwSUof0lox9fp8Ow5EICIN9u0lglu7U"      # reduce_wwPDB_het_dict.tar.gz
}

print("Setting up external tools...")
notebook_utils.setup_external_tools(drive_ids)

print("Configuring Modeller...")
notebook_utils.configure_modeller()

print("✅ Setup Complete!")

## 1. Input Data: Upload Receptor

In [ ]:
#@title Upload Receptor PDB
from google.colab import files
import os
import shutil

print("Upload your Receptor PDB file (e.g., receptor.pdb)")
uploaded = files.upload()
receptor_filename = list(uploaded.keys())[0]
print(f"Receptor uploaded: {receptor_filename}")

# Move to a working directory
os.makedirs("work_dir", exist_ok=True)
shutil.move(receptor_filename, f"work_dir/{receptor_filename}")
receptor_path = f"work_dir/{receptor_filename}"

## 2. Pocket Detection (fpocket)

In [ ]:
#@title Run fpocket and Visualize
import subprocess
import py3Dmol
import ipywidgets as widgets
from IPython.display import display

# Run fpocket
print(f"Running fpocket on {receptor_filename}...")
subprocess.run(f"fpocket -f {receptor_path}", shell=True, check=True)

output_folder = f"{receptor_path}_out"
pockets_dir = os.path.join(output_folder, "pockets")
pockets = [f for f in os.listdir(pockets_dir) if f.endswith(".pdb")]

# Interactive Selector
pocket_dropdown = widgets.Dropdown(
    options=sorted(pockets),
    description='Select Pocket:',
    disabled=False,
)

def view_pocket(pocket_file):
    view = py3Dmol.view(width=800, height=600)
    
    # Load Receptor
    with open(receptor_path, 'r') as f:
        view.addModel(f.read(), "pdb")
    view.setStyle({'cartoon': {'color': 'white'}})
    
    # Load Selected Pocket (Red spheres)
    pocket_full_path = os.path.join(pockets_dir, pocket_file)
    with open(pocket_full_path, 'r') as f:
        view.addModel(f.read(), "pdb")
    view.setStyle({'model': -1}, {'sphere': {'color': 'red', 'opacity': 0.7}})
    
    view.zoomTo()
    view.show()
    return pocket_full_path

widgets.interactive(view_pocket, pocket_file=pocket_dropdown)


## 3. Data Prep & Script Patching

In [ ]:
#@title Prepare Inputs & Patch Scripts
from scripts import notebook_utils

selected_pocket = pocket_dropdown.value
pocket_abs_path = os.path.abspath(os.path.join(pockets_dir, selected_pocket))
print(f"Selected Pocket: {pocket_abs_path}")

# Calculate Box using Utility
center, size = notebook_utils.get_pocket_box(pocket_abs_path)
print(f"Box Center: {center}\nBox Size: {size}")

# Identify paths
notebook_dir = os.getcwd()
repo_dir = os.path.join(notebook_dir, "FrankPEPstein")
scripts_dir = os.path.join(repo_dir, "scripts")
utilities_dir = os.path.join(repo_dir, "utilities")
db_dir = os.path.join(repo_dir, "DB")

# Define Replacements
path_replacements = {
    "/home/jgutierrez/scripts/": f"{scripts_dir}/",
    "/home/jgutierrez/utilities/./vina_1.2.4_linux_x86_64": "vina", 
    "/home/jgutierrez/scripts/reduce_wwPDB_het_dict.txt": f"{db_dir}/reduce_wwPDB_het_dict.txt",
    "/home/jgutierrez/FrankPEPstein/filtered_DB_P5-15_R30_id10": f"{db_dir}/minipockets_surface80_winsize3_size3_curated-db"
}

# Patch Scripts using Utility
notebook_utils.patch_scripts(scripts_dir, path_replacements)


## 4. Pipeline Execution

In [ ]:
#@title Step 4.1: Superposer
import shutil

# Setup working directory for run
run_dir = "run_frankpepstein"
os.makedirs(run_dir, exist_ok=True)
os.chdir(run_dir)

# Copy inputs
shutil.copy(receptor_path, "receptor.pdb")
shutil.copy(pocket_abs_path, "pocket.pdb")

# Define arguments
pepbdb_path = f"{db_dir}/minipockets_surface80_winsize3_size3_curated-db/" # Verify this path after extraction
minipockets_folder = f"{db_dir}/minipockets_surface80_winsize3_size3_curated-db/minipockets/" # Guessing structure

# Run Superposer
cmd_superposer = f"python {scripts_dir}/superposerV5.2_leave1out.py -T pocket.pdb -d {pepbdb_path} -r 0.1 -t 2 -a 3 -fm {minipockets_folder} -x_center {center[0]} -y_center {center[1]} -z_center {center[2]} -x_size {size[0]} -y_size {size[1]} -z_size {size[2]}"
print("Executing Superposer...")
# !{cmd_superposer}
print("Command to run:", cmd_superposer)
# IMPORTANT: Real execution requires the huge DB extracted properly. Dry run for now.

In [ ]:
#@title Step 4.2: FrankVINA Ranking
# Assuming superposer output is ready
print("Running FrankVINA FNKPSTN...")
# !python {scripts_dir}/frankVINA_FNKPSTN.py receptor.pdb 2

In [ ]:
#@title Step 4.3: Patch Clustering & Refinement
print("Running Patch Clustering...")
# !python {scripts_dir}/patch_clustering_V8.7.py -w 6 -t 2

print("Running FrankVINA V3 Refinement...")
# !python {scripts_dir}/frankVINA_V3.py receptor.pdb 2 10

## 5. View & Download Results

In [ ]:
#@title Download Results
!zip -r frankpepstein_results.zip results_folder/
files.download('frankpepstein_results.zip')